# Dataset Cleaning Pipeline
**channels:**
1. Load Excel source files and filter channels (>500 subs, right channel types, valid `mis` flag)
2. Merge and save channel JSON files
3. Verify channels dataset integrity
4. Save final channels dataset

**posts:**

5. Check and fix post JSON filenames
6. Merge all post JSONs into one file
7. Clean post fields
8. Pseudonymize commenters
9. Verify media hash–to–media_id mapping


In [2]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import numpy as np
import json
from pathlib import Path
if 'notebooks' in os.listdir('../'):
    os.chdir('../')

---
# PART 1 — CHANNELS

## Step 1 - Load Excel source files and filter channels
Keep only channels with **>500 subscribers** from `TGStat_merged.xlsx` (news + politics) and exclude those classified as not politics.

In [12]:
# Load TGStat news file and show channels with >=500 subs
TGstat_news = pd.read_excel("../data/TGstat_results_news.xlsx")
print(f"{len(TGstat_news[TGstat_news.n_sub >= 500])} news channels with more than 500 subs")

1396 news channels with more than 500 subs


In [23]:
# Load TGStat politics file
TGstat_results_politics = pd.read_excel('../data/TGstat_results_politics.xlsx')

In [27]:
# Count valid politics channels (exclude == 0 means keep)
print(f"{len(TGstat_results_politics[TGstat_results_politics.exclude == 0])} political channels to keep")
political_channels_to_exclude = list(TGstat_results_politics[TGstat_results_politics.exclude == 1].handle)
print(f"{len(political_channels_to_exclude)} political channels to exclude")

210 political channels to keep
191 political channels to exclude


In [33]:
# Load Amir's channel classification file (with channel_type column)
channels_final_dataset_w_channel_type = pd.read_excel('../data/channels_all_classified_by_Amir.xlsx')
print(channels_final_dataset_w_channel_type.channel_type.unique())

['instant_news' 'politics' 'official_in' 'official_out' nan]


In [34]:
# final dataset should have a defined channel_type (not nan) and not be excluded out of politics. 

In [35]:
channels_final_dataset_w_channel_type = channels_final_dataset_w_channel_type[
    ~channels_final_dataset_w_channel_type['channel_type'].isna()]
channels_final_dataset_w_channel_type = channels_final_dataset_w_channel_type[
    ~channels_final_dataset_w_channel_type['url'].isin(political_channels_to_exclude)
]

In [36]:
len(channels_final_dataset_w_channel_type)

2190

In [37]:
# Re-save with clean index
channels_final_dataset_w_channel_type.to_excel('../data/channels_final_dataset_w_channel_type.xlsx')

## Step 2 - Load and merge channel JSON files
Read `TGstat_channels.json` and `recommended_channels.json`, tag each with a `type` field, concatenate, and save.

In [38]:
with open("../data/Telegram_data/metadata/TGstat_channels.json") as f:
    TGstat_channels = json.load(f)
with open("../data/Telegram_data/metadata/recommended_channels.json") as f:
    recommended_channels = json.load(f)

In [39]:
# Tag each channel with its source type
for i, _ in enumerate(TGstat_channels):
    TGstat_channels[i]["source"] = "TGstat"
for i, _ in enumerate(recommended_channels):
    recommended_channels[i]["source"] = "recommended"

In [40]:
channels = TGstat_channels + recommended_channels
print(f"Total channels: {len(channels)}  (TGstat: {len(TGstat_channels)}, recommended: {len(recommended_channels)})")
channels[-1]

Total channels: 2335  (TGstat: 1741, recommended: 594)


{'id': 1455521249,
 'title': 'کانال خبری پدافند',
 'username': 'Modafeane_Aja',
 'broadcast': True,
 'verified': False,
 'restricted': False,
 'scam': False,
 'fake': False,
 'access_hash': '4117757331980452103',
 'date': '2020-03-17T10:01:51+00:00',
 'about': 'ما را در شبکه های اجتماعی دنبال کنید\n https://padafandnews.com\n https://instagram.com/padafand_aja',
 'subscribers': 1231,
 'channel_url': 'modafeane_aja',
 'restriction_reason': None,
 'type': 'recommended'}

In [ ]:
with open("../data/Telegram_data/metadata/all_channels_w_all_info.json", 'w') as f:
    json.dump(channels, f, ensure_ascii=False, indent=2)

## Step 3 - Verify data integrity
Check for duplicates and missing values in the filtered channel file.

In [41]:
channels_df = pd.read_json("../data/Telegram_data/metadata/all_channels_w_all_info.json")
channels_df['id'] = (channels_df['id'].astype('Int64').astype(str).replace("<NA>", np.nan))

print("channels.json length:\t", len(channels_df))
print("unique username:\t", len(channels_df['username'].unique()))
print("unique channel_url:\t", len(channels_df['channel_url'].unique()))
print("unique id:\t\t", len(channels_df['id'].unique()))
print("channels with id=NaN:\t", len(channels_df[channels_df['id'].isna()]))

channels.json length:	 2198
unique username:	 2159
unique channel_url:	 2198
unique id:		 2191
channels with id=NaN:	 8


In [42]:
# verify: no duplicated channel ids, except those we could not retrieve (id = nan) ✅
channels_df[channels_df.duplicated(subset=['id'], keep=False)]['id'].unique()

array([nan], dtype=object)

In [43]:
# verify: no duplicated channel_urls ✅
channels_df[channels_df.duplicated(subset=['channel_url'], keep=False)]['channel_url'].unique()

array([], dtype=object)

In [44]:
# verify: channels with no title are those we could not retrieve (id always NaN too) ✅
channels_df[channels_df['title'].isna()]['id'].unique()

array([nan], dtype=object)

In [45]:
# verify: channels with no subscribers are those we could not retrieve ✅
channels_df[channels_df.subscribers.isna()]['id'].unique()

array([nan], dtype=object)

In [46]:
# Verify the boolean fields we can safely drop

# broadcast — should be all True (broadcast-only channels)
print("broadcast values:", channels_df['broadcast'].unique())
display(channels_df[channels_df['broadcast'] == 0])  # expect empty

# scam — should be all False
print("scam values:", channels_df['scam'].unique())

# fake — should be all False
print("fake values:", channels_df['fake'].unique())
display(channels_df[channels_df['fake'] == 1])

# restricted — some channels restricted for local law violations
print("restricted values:", channels_df['restricted'].unique())
restricted = channels_df[channels_df['restricted'] == 1]
if len(restricted):
    print([r[0]['text'] for r in restricted['restriction_reason'].values])
    print(f"{len(restricted)} channels were restricted because they violated local laws.")

# verified
print("verified — True:", len(channels_df[channels_df['verified'] == 1]),
      "/ False:", len(channels_df[channels_df['verified'] == 0]))

broadcast values: [ 1. nan]


,id,title,username,broadcast,verified,restricted,scam,fake,access_hash,date,about,subscribers,channel_url,restriction_reason,type,error


scam values: [ 0. nan]
fake values: [ 0. nan]


,id,title,username,broadcast,verified,restricted,scam,fake,access_hash,date,about,subscribers,channel_url,restriction_reason,type,error


restricted values: [ 0.  1. nan]
['This channel can’t be displayed because it violated local laws (France).', 'This channel can’t be displayed because it violated local laws (France).', 'This channel can’t be displayed because it violated local laws (France).', 'This channel can’t be displayed because it violated local laws (France).', 'This channel can’t be displayed because it violated local laws (France).']
5 channels were restricted because they violated local laws.
verified — True: 39 / False: 2151


## Step 4 - Prepare and save final channel dataset
Keep only the useful columns, merge the Amir channel-type classification, rename columns, and export.

In [47]:
# Drop channels we could not retrieve (id = NaN)
print(len(channels_df))
channels_df = channels_df.dropna(subset='id')
print(len(channels_df))

2198
2190


In [58]:
# Keep only relevant columns (drop broadcast, scam, fake, access_hash, username, restriction_reason, error)
channels_df = channels_df[['id', 'title', 'channel_url', 'date', 'about', 'subscribers', 'type', 'verified', 'restricted']]

In [59]:
# Add channel_type from Amir's classification
channels_final_dataset_w_channel_type['id'] = channels_final_dataset_w_channel_type['id'].apply(str)
channels_df = channels_df.merge(channels_final_dataset_w_channel_type[['id', 'channel_type']])

In [60]:
# Rename columns for clarity
channels_df.rename(columns={'channel_url': 'url', 'type': 'source'}, inplace=True)

In [61]:
# Export final channel dataset
channels_df.to_json('../data/Telegram_data/metadata/channels_final_dataset.json',
                    force_ascii=False, orient='records', indent=2)
channels_df.to_csv('../data/Telegram_data/metadata/channels_final_dataset.csv')
print(f"Saved {len(channels_df)} channels to channels_final_dataset.json / .csv")

Saved 2190 channels to channels_final_dataset.json / .csv


---
# PART 2 — POSTS

## Step 5 - Check and fix post JSON filenames, remove posts from removed channels
Make sure each `telegram_messages-<handle>.json` file is named after the canonical `channel_url`, not a former username or title.

In [106]:
# Reload channel metadata (fresh copy with all channels)
channels_df = pd.read_json("../data/Telegram_data/metadata/all_channels_w_all_info.json")
channels_df['id'] = (channels_df['id'].astype('Int64').astype(str).replace("<NA>", np.nan))
channels_df = channels_df.dropna(subset='id')
len(channels_df)

2190

In [107]:
channel_filenames = [f.replace('telegram_messages-', '').replace('.json', '')
                     for f in os.listdir('../data/Telegram_data/posts') if ".json" in f]
print(f"{len(channel_filenames)} post files found")

2190 post files found


In [108]:
# Build lookup dictionaries
username2handle = dict(zip(channels_df['username'], channels_df['channel_url']))
title2handle    = dict(zip(channels_df['title'],    channels_df['channel_url']))
sender_id2handle = dict(zip(channels_df['id'],      channels_df['channel_url']))

def get_sender_id(filename):
    with open(f"../data/Telegram_data/posts/telegram_messages-{filename}.json") as f:
        data = json.load(f)
    if len(data) > 0:
        return int(str(data[0]['sender_id']).replace('-100', ''))
    return None

In [109]:
incorrect_names = []
changed_names   = []

for f in channel_filenames:
    f_ = username2handle.get(f, f)
    f_ = title2handle.get(f_, f_)
    if f_ not in list(channels_df['channel_url']):
        incorrect_names.append(f_)
    elif f != f_:
        changed_names.append((f, f_))

print("changed_names:",  len(changed_names))
print("incorrect_names:", len(incorrect_names))

changed_names: 0
incorrect_names: 0


In [110]:
# Try to resolve truly incorrect names via sender_id lookup
for f in incorrect_names:
    print()
    print(f)
    sender_id = get_sender_id(f)
    print(sender_id)
    handle = sender_id2handle.get(sender_id, None)
    if handle:
        print(f'Renaming "telegram_messages-{f}.json" → "telegram_messages-{handle}.json"')
        try:
            os.rename(
                f"../data/Telegram_data/posts/telegram_messages-{f}.json",
                f"../data/Telegram_data/posts/telegram_messages-{handle}.json"
            )
        except FileNotFoundError:
            print("already modified")

In [111]:
# remove channels than are not in channels_final_dataset
removed_channels = set(channel_filenames) - set(channels_df['channel_url']).union(set(channels_df['username']))

In [102]:
for f in removed_channels:
    os.rename(
                f"../data/Telegram_data/posts/telegram_messages-{f}.json",
                f"../data/Telegram_data/posts/removed_channels/telegram_messages-{f}.json"
            )

In [112]:
# Rename files that used a username instead of channel_url
for f in changed_names:
    print(f)
    print(f'Renaming "telegram_messages-{f[0]}.json" → "telegram_messages-{f[1]}.json"')
    try:
        os.rename(
            f"../data/Telegram_data/posts/telegram_messages-{f[0]}.json",
            f"../data/Telegram_data/posts/telegram_messages-{f[1]}.json"
        )
    except FileNotFoundError:
        print("already modified")

## Step 6 - Merge all post JSONs into one file
Iterate over all `telegram_messages-*.json` files (TGstat + recommended) and combine them into a single dict keyed by `channel_url`. Each post gets a unique `uid`.

In [115]:
BASE_DIR = Path("../data/Telegram_data/posts")

output = {}
uid_counter = 1

for fname in tqdm(os.listdir(BASE_DIR)):
    if not fname.startswith("telegram_messages-") or not fname.endswith(".json"):
        continue

    channel_url = fname.replace("telegram_messages-", "").replace(".json", "")

    with open(BASE_DIR / fname, "r", encoding="utf-8") as f:
        posts = json.load(f, parse_int=str)

    if channel_url not in output:
        output[channel_url] = []

    for post in posts:
        post["uid"] = str(uid_counter)
        uid_counter += 1
        if "id" in post:
            post["id"] = str(post["id"])
        output[channel_url].append(post)

with open("../data/Telegram_data/all_posts.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print(f"✅ Done. Total posts: {uid_counter - 1}")

  0%|          | 0/2192 [00:00<?, ?it/s]

✅ Done. Total posts: 3653769


## Step 7 - Clean post fields
Normalise `document_id` / `photo_id` → `media_id` and keep only the relevant columns.

**Post schema**
| Field | Notes |
|---|---|
| `id`, `uid`, `sender_id`, `date`, `text` | core fields |
| `views`, `forwards`, `reactions`, `comments` | engagement |
| `origin_from_id`, `origin_channel_post` | forwarded-post fields |
| `media_id`, `media_type`, `size` | media fields |

In [116]:
with open("../data/Telegram_data/all_posts.json") as f:
    all_posts = json.load(f)

In [117]:
columns = ['id', 'uid', 'sender_id', 'date', 'text', 'views', 'forwards',
           'reactions', 'comments', 'origin_from_id', 'origin_channel_post',
           'media_id', 'media_type', 'size']

all_posts_final = dict()

for channel, posts in tqdm(all_posts.items()):
    all_posts_final[channel] = []
    for post in posts:
        if 'document_id' in post:
            post['media_id'] = post['document_id']
        if 'photo_id' in post:
            post['media_id'] = post['photo_id']
        row_dict = {col: post.get(col) for col in columns}
        all_posts_final[channel].append(row_dict)

with open("../data/Telegram_data/all_posts.json", 'w') as f:
    json.dump(all_posts_final, f, indent=2, ensure_ascii=False)

print(f"Cleaned posts for {len(all_posts_final)} channels")

  0%|          | 0/2190 [00:00<?, ?it/s]

Cleaned posts for 2190 channels


## Step 8 - Pseudonymize commenter IDs
Replace each real commenter ID with a stable, anonymous integer index.

In [119]:
#with open('../data/Telegram_data/posts.json') as f:
#    all_posts_final = json.load(f)

print(f"{len(all_posts_final)} channels loaded")

2190 channels loaded


In [120]:
# Collect all commenter IDs
commenters_ids = []
for k, posts in tqdm(all_posts_final.items()):
    for post in posts:
        commenters_ids += post['comments']

commenters_ids = [c['id'] for c in commenters_ids]
print(f"Total comments: {len(commenters_ids)}")
print(f"Unique commenter IDs: {len(set(commenters_ids))}")

  0%|          | 0/2190 [00:00<?, ?it/s]

Total comments: 3197201
Unique commenter IDs: 2159866


In [121]:
# Build stable pseudo-ID mapping (sorted so mapping is reproducible)
unique_commenter_ids = sorted(list(set(commenters_ids)))
commenters_id2pseudo_id = {c_id: i for i, c_id in enumerate(unique_commenter_ids)}

# Save the mapping for reference
with open('../data/commenters_id2pseudo_id.json', 'w') as f:
    json.dump(commenters_id2pseudo_id, f, indent=2, ensure_ascii=False)

In [122]:
# Apply pseudonymization in-place
for channel, posts in tqdm(all_posts_final.items(), total=len(all_posts_final)):
    posts_new = []
    for post in posts:
        post_new = dict(post)
        comments = post.get("comments") or []
        if comments:
            new_comments = []
            for comment in comments:
                c = dict(comment)
                orig_id = c.get("id")
                if orig_id is not None:
                    orig_id = str(orig_id)
                    c["id"] = commenters_id2pseudo_id[orig_id]
                new_comments.append(c)
            post_new["comments"] = new_comments
        posts_new.append(post_new)
    all_posts_final[channel] = posts_new

print(f"Total unique commenters pseudonymized: {len(commenters_id2pseudo_id)}")

  0%|          | 0/2190 [00:00<?, ?it/s]

Total unique commenters pseudonymized: 2159866


In [123]:
# Quick verification: collect pseudo IDs from the updated posts
comments_pseudo = []
for k, posts in tqdm(all_posts_final.items()):
    for post in posts:
        comments_pseudo += post['comments']

all_ids = [c['id'] for c in comments_pseudo]
print(f"Total comments:          {len(all_ids)}")
print(f"Unique pseudo-IDs:       {len(set(all_ids))}")
print(f"All IDs are integers:    {all(isinstance(i, int) for i in all_ids)}")

  0%|          | 0/2190 [00:00<?, ?it/s]

Total comments:          3197201
Unique pseudo-IDs:       2159866
All IDs are integers:    True


In [124]:
# Save final pseudonymized posts file
with open('../data/Telegram_data/posts_pseudo.json', 'w') as f:
    json.dump(all_posts_final, f, indent=2, ensure_ascii=False)

print("✅ posts_pseudo.json saved")

✅ posts_pseudo.json saved


## Step 9 - Verify media hash ↔ media_id mapping
Each `media_id` should map to exactly one `content_hash`. If a hash has multiple `media_id`s, the same file was re-uploaded to Telegram.

In [ ]:
posts_w_hash = {}
for filename in tqdm(os.listdir("api_attribution_media_updated_by_collect/1/")):
    if Path(filename).suffix == ".json":
        with open(Path("api_attribution_media_updated_by_collect/1/") / filename) as f:
            channel_postids = json.load(f)
        posts_w_hash.update(channel_postids)

columns_hash = ["channel_url", "id", "mime_type", "size_bytes", "content_hash"]
rows_hash = []
for channel, posts in tqdm(posts_w_hash.items()):
    for post in posts:
        row_dict = {col: post.get(col) for col in columns_hash}
        row_dict["channel_url"] = channel
        rows_hash.append(row_dict)

posts_w_hash_df = pd.DataFrame(rows_hash)
print(posts_w_hash_df.shape)

In [ ]:
# Reload all_posts to get the full post metadata for the channels with hashes
with open("../data/Telegram_data/all_posts.json") as f:
    all_posts = json.load(f)

channels_oi = set(posts_w_hash_df.channel_url)
subset_posts = {channel: all_posts[channel] for channel in all_posts if channel in channels_oi}

In [ ]:
# Extract video posts from the subset
video_cols = ['id', 'has_media', 'media_type', 'media_id']
rows_video = []
for channel, posts in tqdm(subset_posts.items()):
    for post in posts:
        if 'document_id' in post:
            post['media_id'] = post['document_id']
        if 'photo_id' in post:
            post['media_id'] = post['photo_id']
        row_dict = {col: post.get(col) for col in video_cols}
        row_dict["channel_url"] = channel
        rows_video.append(row_dict)

channels_origin_posts_df = pd.DataFrame(rows_video)
channels_origin_posts_df = channels_origin_posts_df[channels_origin_posts_df['media_type'] == 'video'].copy()
channels_origin_posts_df['id'] = channels_origin_posts_df['id'].apply(int)
posts_w_hash_df['id'] = posts_w_hash_df['id'].apply(int)

In [ ]:
# Merge and check: how many content_hashes map to more than one media_id?
merged = channels_origin_posts_df.merge(posts_w_hash_df, on=['channel_url', 'id'])
merged_by_content_hash = merged.groupby('content_hash')['media_id'].unique().reset_index()
merged_by_content_hash['n_media_id'] = merged_by_content_hash['media_id'].apply(len)

multi = merged_by_content_hash[merged_by_content_hash.n_media_id > 1]
print(f"{len(multi)} hashes map to more than one media_id (same video re-uploaded)")
display(merged_by_content_hash[merged_by_content_hash.n_media_id > 1].head())